In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.options.display.max_rows = 999
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

In [2]:
FeatureTrain = pd.read_csv('./data/dengue_features_train.csv',parse_dates = ['week_start_date'])
TargetTrain = pd.read_csv('./data/dengue_labels_train.csv' )
FeatureTest = pd.read_csv('./data/dengue_features_test.csv',parse_dates = ['week_start_date'])
Answersheet = pd.read_csv('./data/submission_format.csv' )

#                                  Presentation Data Exploration

In [3]:
panel = TargetTrain.merge(FeatureTrain,on = ['city','year','weekofyear'] ,how = 'outer') 
 
kelvin = ['reanalysis_air_temp_k', 'reanalysis_avg_temp_k',   \
          'reanalysis_max_air_temp_k','reanalysis_min_air_temp_k','reanalysis_dew_point_temp_k']
panel.loc[:,kelvin] = panel.loc[:,kelvin]-273.15# kelvin to C
FeatureTest.loc[:,kelvin] =FeatureTest.loc[:,kelvin]-273.15# kelvin to C
panel.columns
timeid = ['year', 'weekofyear']
green = ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw']
precipitation  =['precipitation_amt_mm','reanalysis_sat_precip_amt_mm','station_precip_mm','reanalysis_precip_amt_kg_per_m2',]
avg_temp = ['reanalysis_air_temp_k', 'reanalysis_avg_temp_k',  'station_avg_temp_c' ]
min_temp = ['station_min_temp_c','reanalysis_min_air_temp_k']
max_temp = ['station_max_temp_c','reanalysis_max_air_temp_k']
dtr =  ['reanalysis_tdtr_k', 'station_diur_temp_rng_c']
humid = ['reanalysis_dew_point_temp_k','reanalysis_specific_humidity_g_per_kg', 'reanalysis_relative_humidity_percent']
selected = humid + dtr + max_temp +min_temp +avg_temp +precipitation +timeid+green + ['city','total_cases','week_start_date']
features_selected= humid + dtr + max_temp +min_temp +avg_temp +precipitation +timeid+green  
panel = panel.loc[:,selected]

 
 


In [4]:
classes = ['reanalysis_dew_point_temp_k','reanalysis_max_air_temp_k','reanalysis_min_air_temp_k',\
           'green_s','green_n','precipitation_amt_mm']

In [5]:
 
def get_data_panel(panel,city_name):
    panel_train =  panel.loc[panel.city == city_name].interpolate()  .set_index('week_start_date').copy().drop(['city'],axis = 1).copy()
    
    panel_test = FeatureTest.loc[FeatureTest.city == city_name].interpolate().set_index('week_start_date').copy().drop('city',axis = 1)
    panel_test['total_cases'] = np.nan 
    train_len = len(panel_test['total_cases'])
    panel = pd.concat([panel_train,panel_test],sort = False)
    
    
    panel['green_s']  = panel.loc[:,green[:2]].mean(1)
    panel['green_n']  = panel.loc[:,green[2:]].mean(1) 
    panel['green']  = panel.loc[:,green ].mean(1) 

    panel  = panel 
    return panel,train_len

panel_sj,len_sj = get_data_panel(panel,'sj')
panel_iq,len_iq = get_data_panel(panel,'iq')

def min_max(data):
     
    return (data - data.min())/(data .max()- data.min())



from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

drops = ['reanalysis_specific_humidity_g_per_kg']  + green +\
['reanalysis_sat_precip_amt_mm','reanalysis_avg_temp_k','year','weekofyear','total_cases'] 

def append_shift(left,right,selected,shift,smooth =52):
    new_names = []
    right = right.copy().apply(lambda x : x.clip(x.quantile(0.05),x.quantile(0.95)),axis = 0)
    for name in selected:
        new_name = name + '_shift_' +str(shift)+'_smooth_' + str(smooth)
        new_names.append(new_name)
 
        left[new_name] = right.rolling(smooth     ).mean().shift(shift).loc[:,name ]
    return left

def append_diff(left,right,selected,diff,smooth =52):
    new_names = []
    right = right.copy().apply(lambda x : x.clip(x.quantile(0.05),x.quantile(0.95)),axis = 0)
    for name in selected:
        new_name = name + '_diff_'  +'_smooth_' + str(smooth)
        new_names.append(new_name)
 
        left[new_name] = right.diff(1).rolling(smooth   ).mean().loc[:,name ]
    return left
 
def de_season_tri(series):
    
    target = series.name
    series = pd.DataFrame(series.copy())
    series["season_sin"] = np.sin(series.index.weekofyear/53*6.2831)
    series["season_cos"] = np.cos(series.index.weekofyear/53*6.2831)


    X_out = series.drop(target,axis = 1) 
    X_out  = sm.add_constant(X_out) 




    train_df = series.dropna()
    train_y = train_df  .loc[:,target]
    train_y =train_y.clip(train_y.quantile(0.05),train_y.quantile(0.95))
    train_X = train_df  .drop(target,axis = 1)
    train_X = sm.add_constant(train_X)
    model = sm.OLS(train_y,train_X)
    res = model.fit()
    return  series.loc[:,target] .ravel()/res  .predict(X_out) .ravel() - 1


def get_season_tri(series):
    
    target = series.name
    series = pd.DataFrame(series.copy())
    series["season_sin"] = np.sin(series.index.weekofyear/53*6.2831)
    series["season_cos"] = np.cos(series.index.weekofyear/53*6.2831)


    X_out = series.drop(target,axis = 1) 
    X_out  = sm.add_constant(X_out) 




    train_df = series.dropna()
    train_y = train_df  .loc[:,target]
    train_y =train_y.clip(train_y.quantile(0.05),train_y.quantile(0.95))
    train_X = train_df  .drop(target,axis = 1)
    train_X = sm.add_constant(train_X)
    model = sm.OLS(train_y,train_X)
    res = model.fit()
    return   res  .predict(X_out) .ravel()  



 

panel_sj,len_sj = get_data_panel(panel,'sj')
 
panel_iq,len_iq = get_data_panel(panel,'iq')
 

In [6]:
class dataset_class:
    """
    dataset iterator 
    """
    def __init__(self,inputs,targets):
        self.inputs = inputs 
        self.targets = targets 
        self.limit   = len(targets)
        
        
    def __iter__(self): 
        self.counter = 0 
        return self
    
    def __next__(self): 
  
        # Store current value ofx 
        self.counter  
  
   
        if self.counter >= self.limit: 
            raise StopIteration 
        x = self.counter
        # Else increment and return old value 
        
        self.counter  = self.counter + 1 
        return self.inputs[x],self.targets[x] 
    
    def __len__(self):
        # Return the size of the dataset
        return len(self.targets)

    def __getitem__(self, index):
        # Retrieve inputs and targets at the given index
        X = self.inputs[index]
        y = self.targets[index]

        return X, y
    
def make_samples(train_X,train_y,size):
    Xs,Ys = [],[]
    
    while(len(Xs) <size):
        start = np.random.randint(52)
        while(start < len(train_X)):
            if start + 52 < len(train_X):
                Xs.append(train_X.iloc[start:start + 52].values)
                Ys.append(train_y.iloc[start:start + 52].values)
            else:
                break
            start += 52
    return Xs,Ys

In [7]:
panel_sj_selected  = panel_sj.loc[:,['total_cases']]
panel_iq_selected  = panel_iq.loc[:,['total_cases']] 
np.random.seed(10)
def data_process_random_forest(panel,panel_candidate ):
    
    """
    Make sure it is at least forward filled
    """
    panel = append_shift(panel,panel_candidate,\
          panel_candidate.columns,0,52)
    
    panel = append_shift(panel,panel_candidate,\
          panel_candidate.columns,0,20)
    
    panel = append_diff(panel,panel_candidate,\
          panel_candidate.columns,26,26)
    
    
    # de - seasons
    de_seasons = panel_candidate.apply(de_season_tri,axis = 0)
    new_names = [name +'_ds' for name in de_seasons.columns]
    de_seasons.columns = new_names
    
    
    panel = append_shift(panel,de_seasons,\
          de_seasons.columns,0,52)
    
    panel = append_shift(panel,de_seasons,\
          de_seasons.columns,0,10)
    
    panel = append_diff(panel,de_seasons,\
          de_seasons.columns,26,30)
 
    return panel .fillna(method = 'ffill')

panel_sj_selected =  data_process_random_forest(panel_sj_selected,panel_sj.drop(drops,axis = 1))
panel_iq_selected =  data_process_random_forest(panel_iq_selected,panel_iq.drop(drops,axis = 1)) 
#panel_sj_selected['season'] = get_season_tri(panel_sj_selected.total_cases )
#panel_iq_selected['season'] = get_season_tri(panel_iq_selected.total_cases )
 


D:\Anaconda3\envs\interview_env\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [8]:
selected = ['station_min_temp_c_ds_shift_0_smooth_10',
       'reanalysis_precip_amt_kg_per_m2_ds_shift_0_smooth_52',
       'reanalysis_tdtr_k_ds_shift_0_smooth_52', 'green_ds_diff__smooth_30',
       'green_diff__smooth_26', 'green_n_diff__smooth_26',
       'green_n_ds_diff__smooth_30', 'green_s_ds_diff__smooth_30',
       'reanalysis_precip_amt_kg_per_m2_ds_diff__smooth_30',
       'station_precip_mm_ds_diff__smooth_30',
       'precipitation_amt_mm_ds_diff__smooth_30',
       'reanalysis_air_temp_k_ds_diff__smooth_30',
       'reanalysis_min_air_temp_k_ds_diff__smooth_30',
       'station_diur_temp_rng_c_ds_diff__smooth_30',
       'reanalysis_relative_humidity_percent_ds_diff__smooth_30',
       'green_s_diff__smooth_26', 'station_precip_mm_diff__smooth_26']

In [17]:
import statsmodels.api as sm
def normalize(df, how ):
    if how == 'z':
        return (df - df.mean())/(df.std())
    if how == 'r':
        return (df - df.min())/(df.max() - df.min())
    
    
    
train_y_sj, train_X_sj =   panel_sj_selected  .iloc[: -len_sj].dropna().loc[:,'total_cases'],\
                             normalize(panel_sj_selected,'z' ).iloc[: -len_sj] .dropna() .loc[:,selected]
 
train_y_iq, train_X_iq =   panel_iq_selected .iloc[60: -len_iq].dropna() .loc[:,'total_cases'],\
                             normalize(panel_iq_selected,'z').iloc[60: -len_iq] .dropna() .loc[:,selected]
                         
 

test_X_sj = panel_sj_selected  .iloc[-len_sj: ]
test_X_iq = panel_iq_selected   .iloc[-len_iq: ]

train_Xs,train_Ys =  make_samples(train_X_sj.iloc[ :500,1:].loc[:,selected].dropna(axis = 1),train_y_sj.iloc[:500 ],40)
validate_Xs,validate_Ys =  make_samples(train_X_sj.iloc[500: ,1:].loc[:,selected].dropna(axis = 1),train_y_sj.iloc[500:  ],10)
training_set = dataset_class(train_Xs,train_Ys)
validation_set = dataset_class(validate_Xs,validate_Ys ) 

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
feature_size = len(selected)-1
hidden_size = 5
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Recurrent layer
        # YOUR CODE HERE!
        self.lstm = nn.LSTM(input_size=feature_size,
                         hidden_size=hidden_size,
                         num_layers=1,
                         bidirectional=False)
        
        # Output layer
        self.l_out = nn.Linear(in_features=hidden_size,
                            out_features=1)
        
    def forward(self, x):
        # RNN returns output and last hidden state
        x, (h, c) = self.lstm(x)
        
        # Flatten output for feed-forward layer
        x = x.view(-1, self.lstm.hidden_size)
        
        # Output layer
        x = self.l_out(x)
        
        return x

net = Net()

print(net)

Net(
  (lstm): LSTM(16, 5)
  (l_out): Linear(in_features=5, out_features=1, bias=True)
)


In [19]:

import warnings
warnings.filterwarnings('ignore')

num_epochs = 200
net = Net()
criterion = nn.L1Loss()

training_loss, validation_loss = [], []
optimizer = torch.optim.Adam(net.parameters(), lr=3e-4) 

In [21]:


for epoch in range(num_epochs ) :

    epoch_training_loss = 0
    epoch_validation_loss = 0

    net.eval()
    for inputs, targets in validation_set:
        #  validating paries
        inputs = torch.Tensor(np.array([inputs,]) ).permute(1,0,2)
        targets = torch.LongTensor(targets)
        
        outputs = net.forward(inputs)
        loss = criterion(outputs, targets)    
        epoch_validation_loss += loss.detach().numpy()
   

    net.train()
    for inputs, targets in training_set:

        inputs = torch.Tensor(np.array([inputs ]) ).permute(1,0,2)
        targets = torch.LongTensor(targets) .view(len(targets),1)
        outputs = net.forward(inputs)
        loss = criterion(outputs, targets) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_training_loss += loss.detach().numpy()
    training_loss.append(epoch_training_loss/len(training_set))
     
    validation_loss.append(epoch_validation_loss/len(validation_set))

    # early stop 
    if epoch % 5 == 0 and  validation_loss[-1] > validation_loss[-5]:
        break
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')